In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from ultralytics import YOLO
from PIL import Image
import requests
from io import BytesIO
import math
import cv2
print("Hello Word")

Hello Word


In [8]:
vid = cv2.VideoCapture('CorridorTest4.mp4')
import math
intersections=[]
segmented=[]
out = cv2.VideoWriter('corridorOut.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (500,500))

if (vid.isOpened()== False):
    print("Error opening video")

while vid.isOpened():
    ret, frame = vid.read()
    blur = cv2.GaussianBlur(frame,(11,11),0)
    if not ret:
      break
    
    # img = cv2.imread('/content/corridor1.jpg')
    else:
      dim = (500 , 500)
      img = cv2.resize(frame, dim,interpolation = cv2.INTER_AREA)
      gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
      blur = cv2.GaussianBlur(gray, (5, 5), 20)

      edges = cv2.Canny(blur,threshold1 = 0,threshold2 = 50, apertureSize = 3, L2gradient = True)
      kernel = np.ones((3, 3), np.uint8)

      cdst = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

      lines = cv2.HoughLines(edges, 1, np.pi / 180, 150, None, 0, 0)


      res=[]
      for i in lines:
        print(i)
        if i[0][1]>=0 and i[0][1]<=0.1:
          pass
        else:
          if i[0][1]>= 1.5 and i[0][1]<=1.6:
            pass
          else:
            res.append(i)

#https://stackoverflow.com/questions/46565975/find-intersection-point-of-two-lines-drawn-using-houghlines-opencv#:~:text=kmeans()%20with%20theta%20as,store%20those%20and%20use%20them.
      from collections import defaultdict
      def segment_by_angle_kmeans(lines, k=2, **kwargs):

          # Define criteria = (type, max_iter, epsilon)
          default_criteria_type = cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER
          criteria = kwargs.get('criteria', (default_criteria_type, 10, 1.0))
          flags = kwargs.get('flags', cv2.KMEANS_RANDOM_CENTERS)
          attempts = kwargs.get('attempts', 10)

          # returns angles in [0, pi] in radians
          angles = np.array([line[0][1] for line in lines])
          # multiply the angles by two and find coordinates of that angle
          pts = np.array([[np.cos(2*angle), np.sin(2*angle)]
                          for angle in angles], dtype=np.float32)

          # run kmeans on the coords
          labels, centers = cv2.kmeans(pts, k, None, criteria, attempts, flags)[1:]
          labels = labels.reshape(-1)  # transpose to row vec

          # segment lines based on their kmeans label
          segmented = defaultdict(list)
          for i, line in enumerate(lines):
              segmented[labels[i]].append(line)
          segmented = list(segmented.values())
          return segmented
      
      segmented = segment_by_angle_kmeans(res)

      def intersection(line1, line2):

        rho1, theta1 = line1[0]
        rho2, theta2 = line2[0]
        A = np.array([
            [np.cos(theta1), np.sin(theta1)],
            [np.cos(theta2), np.sin(theta2)]
        ])
        b = np.array([[rho1], [rho2]])
        x0, y0 = np.linalg.solve(A, b)
        x0, y0 = int(np.round(x0)), int(np.round(y0))
        return [[x0, y0]]


      def segmented_intersections(lines):

          intersections = []
          for i, group in enumerate(lines[:-1]):
              for next_group in lines[i+1:]:
                  for line1 in group:
                      for line2 in next_group:
                          intersections.append(intersection(line1, line2)) 
          return intersections
    
      intersections = segmented_intersections(segmented)
      Pt1 = []
      Pt2 = []
      for i in intersections:
         Pt1.append(i[0][0])
         Pt2.append(i[0][1])
      
      averageP1 = sum(Pt1) / len(Pt1)
      averageP2 = sum(Pt2) / len(Pt2)

      pointCenter = (int(averageP1), int(averageP2))
      if pointCenter is not None:
        cv2.circle(img, pointCenter, 7, (255, 0, 0), cv2.FILLED)



      if res is not None:
        for i in range(0, len(res)):
          rho = res[i][0][0]
          theta = res[i][0][1]
          a = math.cos(theta)
          b = math.sin(theta)
          x0 = a * rho
          y0 = b * rho
          pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
          pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
          cv2.line(img, pt1, pt2, (0,0,255), 1)

    s=img.shape

    w=s[1]
    h=s[0]

    cv2.line(img, (int(w/2), 0), (int(h/2), h), (0,255,0),2)
    cv2.line(img, (0, int(h/2)), (w, int(w/2)), (0,255,0),2)

    out.write(img)
    cv2.imshow("test", img)
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break

vid.release()
out.release()
cv2.destroyAllWindows()

[[        335     0.90757]]
[[         21      2.2515]]
[[         19      2.2689]]
[[        340     0.89012]]
[[        337     0.87266]]
[[          9      2.2864]]
[[        339     0.90757]]
[[         54      1.5708]]
[[        335     0.90757]]
[[         21      2.2515]]
[[         19      2.2689]]
[[        340     0.89012]]
[[         18      2.2515]]
[[          9      2.2864]]
[[        337     0.87266]]
[[         54      1.5708]]
[[        334     0.90757]]
[[         21      2.2515]]
[[        339     0.89012]]
[[         20      2.2689]]
[[         18      2.2515]]
[[         10      2.2864]]
[[         10      1.5708]]
[[        338     0.90757]]
[[        334     0.90757]]
[[         22      2.2515]]
[[        339     0.89012]]
[[         20      2.2689]]
[[         10      2.2864]]
[[          9      1.5708]]
[[        338     0.90757]]
[[         18      2.2515]]
[[         53      1.5708]]
[[        336     0.87266]]
[[        337     0.92502]]
[[        334     0.